In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import time 
from tqdm import tqdm

### Paths and correlation generating functions

In [2]:
def simulate_correlated_gbm(
    S0,
    r,
    q,
    sigma,
    corr_matrix,
    T,
    n_steps,
    n_sims=1,
    random_seed=None
):
    """
    Simulates correlated GBM paths for multiple assets that each pay a continuous
    dividend yield q[i]. Under the risk-neutral measure, the drift becomes (r - q[i])
    for the i-th asset.

    Parameters
    ----------
    S0 : array-like of shape (n_assets,)
        Initial prices of the assets.
    r : float
        Risk-free interest rate (annualized).
    q : array-like of shape (n_assets,) or float
        Continuous dividend yield(s) for the assets. Can be a single float (applied
        to all assets) or an array specifying each asset's q.
    sigma : array-like of shape (n_assets,)
        Volatilities of the assets.
    corr_matrix : 2D array-like of shape (n_assets, n_assets)
        Correlation matrix among the assets (must be positive semi-definite).
    T : float
        Total time horizon for the simulation (in years).
    n_steps : int
        Number of time steps in the simulation.
    n_sims : int, optional
        Number of Monte Carlo simulation paths. Default is 1.
    random_seed : int, optional
        Random seed for reproducibility. Default is None.

    Returns
    -------
    paths : ndarray, shape (n_sims, n_assets, n_steps+1)
        Simulated price paths for each simulation, for each asset, over the time grid.
        The time grid is [0, T], split into n_steps intervals => (n_steps+1) total points.
    """
    # Convert inputs to NumPy arrays
    S0 = np.array(S0, dtype=np.float64)
    sigma = np.array(sigma, dtype=np.float64)
    corr_matrix = np.array(corr_matrix, dtype=np.float64)

    n_assets = len(S0)

    # If q is a scalar, make it an array of length n_assets
    if isinstance(q, (int, float)):
        q = np.full(n_assets, float(q))
    else:
        q = np.array(q, dtype=np.float64)
    if q.shape[0] != n_assets:
        raise ValueError("If 'q' is an array, it must have the same length as 'S0'.")

    # Set the random seed if provided
    if random_seed is not None:
        np.random.seed(random_seed)

    # Time increment
    dt = T / n_steps

    # Cholesky decomposition of the correlation matrix
    L = np.linalg.cholesky(corr_matrix)

    # Pre-allocate the output array
    paths = np.zeros((n_sims, n_assets, n_steps + 1))
    paths[:, :, 0] = S0  # set initial prices

    # For each simulation
    for sim_idx in range(n_sims):
        for t in range(n_steps):
            # Generate independent standard normals
            epsilon = np.random.normal(size=n_assets)
            # Correlate them
            Z = L @ epsilon

            # drift_term = (r - q - 0.5 * sigma^2) * dt
            drift_term = (r - q - 0.5 * sigma**2) * dt
            diffusion_term = sigma * np.sqrt(dt) * Z

            # Update each asset price
            paths[sim_idx, :, t + 1] = (
                paths[sim_idx, :, t] *
                np.exp(drift_term + diffusion_term)
            )

    return paths


def build_correlation_matrix(rho, n_assets):
    """
    Returns an n_assets x n_assets correlation matrix with rho off-diagonal and 1 on the diagonal.
    """
    # Create the matrix
    corr_matrix = np.full((n_assets, n_assets), rho)  # Fill with rho
    np.fill_diagonal(corr_matrix, 1.0)  # Set diagonal to 1.0

    return corr_matrix


def generate_parameters(n_assets, value=None, mode='same', dist_params=None):
    """
    Generates parameter vectors for `n_assets` based on the specified mode ('same', 'normal', or 'uniform'). 
    Accepts optional `value` and `dist_params` for customization; raises ValueError for invalid inputs.
    """
    if mode == 'same':
        return np.full(n_assets, value)
    elif mode == 'normal':
        return np.random.normal(loc=value, scale=dist_params['scale'], size=n_assets)
    elif mode == 'uniform':
        return np.random.uniform(low=dist_params['low'], high=dist_params['high'], size=n_assets)
    else:
        raise ValueError("Invalid mode. Choose from 'same', 'normal', or 'uniform'.")
    

### Payoff function options


In [ ]:

def max_among_n_assets_each_date(
    n_paths,
    style,
    strike,
    monitoring_dates,
    T
):
    """
    Compute ONE payoff per monitoring date by taking the maximum price 
    across 'n_assets' at that date (i.e., each row in n_paths is a single
    time dimension, and the columns are the assets).
    Returns
    -------
    payoffs : np.ndarray of shape (num_monitoring_dates,)
        A single payoff per monitoring date, computed as:
        payoffs[j] = max(0, (max_asset_price_at_t - strike)) [call]
        or max(0, (strike - max_asset_price_at_t)) [put].
    """
    # Convert inputs to NumPy arrays
    n_paths = np.asarray(n_paths, dtype=float)
    monitoring_dates = np.asarray(monitoring_dates, dtype=float)

    # Validate shape: for a single scenario with n_assets, we expect 2D
    if n_paths.ndim != 2:
        raise ValueError("n_paths must be 2D: (num_timesteps, n_assets).")
    
    num_timesteps, num_assets = n_paths.shape
    
    # Create a time grid [0, T] with num_timesteps points
    time_grid = np.linspace(0, T, num_timesteps)

    # For each monitoring date, find the closest index in time_grid
    monitoring_indices = [np.argmin(np.abs(time_grid - md)) for md in monitoring_dates]

    # Prepare output array: one payoff per monitoring date
    payoffs = np.zeros(len(monitoring_indices), dtype=float)

    # For each monitoring date...
    for j, idx in enumerate(monitoring_indices):
        # Take the max among all assets at that time index
        max_price = np.max(n_paths[idx, :])

        # Compute the payoff for call or put
        if style == 'call':
            payoff = max(0, max_price - strike)
        else:  # 'put'
            payoff = max(0, strike - max_price)

        payoffs[j] = payoff

    return payoffs
    

In [3]:
def payoff_arithmetic_basket(S, K, option_type, weights=None):
    """
    Computes the payoff of an arithmetic basket call or put option.
    Args:
        S (ndarray): 
            2D array of shape (n_sims, n_assets), 
            where each row represents one path's asset prices 
            at maturity (or at some time t).
        K (float): Strike price of the option.
        option_type (str): Either 'call' or 'put'.
        weights (ndarray or None): 
            1D array of shape (n_assets,) indicating the basket weights.
            If None, uniform weights are assumed.
    Returns:
        payoff (ndarray):
            1D array of shape (n_sims,); payoff for each path.
    """
    n_sims, n_assets = S.shape

    # If weights not provided, assume uniform
    if weights is None:
        weights = np.ones(n_assets) / n_assets

    # 1) Compute weighted sum across the assets = basket price
    basket_price = np.sum(S * weights, axis=1)  # shape: (n_sims,)

    # 2) Depending on call or put:
    if option_type.lower() == 'call':
        payoff = np.maximum(basket_price - K, 0.0)
    elif option_type.lower() == 'put':
        payoff = np.maximum(K - basket_price, 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    return payoff

In [4]:
# This is currenly used

def payoff_fun(S_t, K, style):
    """
    Computes the payoff from a multi-asset option that depends
    on the maximum underlying price among all assets.

    Parameters
    ----------
    S_t : ndarray, shape (n_sims, n_assets)
        Asset prices for each simulation at time t.
    K : float
        Strike price.
    style : str
        'call' or 'put'.
    Returns
    -------
    payoff : ndarray, shape (n_sims,)
        The payoff for each simulation path.
    """
    print(S_t.shape)
    # Max price among n_assets
    S_max = np.max(S_t, axis=1)
    if style.lower() == 'call':
        return np.maximum(S_max - K, 0.0)
    elif style.lower() == 'put':
        return np.maximum(K - S_max, 0.0)
    else:
        raise ValueError("style must be either 'call' or 'put'.")


### RLLN helper functions

In [7]:
# 1) Helper Functions for Closed-Form Continuation Value & RLNN


def conditional_expectation_closed_form(mu, sigma):
    """
    Computes E[max(X, 0)] for X ~ Normal(mu, sigma^2).

    Formula: E[max(X,0)] = sigma * phi(mu/sigma) + mu * Phi(mu/sigma),
    where phi and Phi are the standard normal pdf and cdf, respectively.
    """
    if sigma < 1e-14:
        # effectively no volatility -> max(mu, 0)
        return max(mu, 0.0)

    z = mu / sigma
    pdf_val = norm.pdf(z)  # phi(z)
    cdf_val = norm.cdf(z)  # Phi(z)
    return sigma * pdf_val + mu * cdf_val



def build_covariance_matrix_dt(sigma_vector, correlation_matrix, dt):
    """
    Build the *log-price* covariance matrix for the time increment dt.
      - sigma_vector: shape (d,) -> [sigma_1, ..., sigma_d]
      - correlation_matrix: shape (d,d)
      - dt: float, (t_m - t_{m-1})
    Returns:
      covariance_matrix: shape (d,d)
    """
    diag_sigma = np.diag(sigma_vector)            # shape (d, d)
    base_cov   = diag_sigma @ correlation_matrix @ diag_sigma  # shape (d, d)
    return base_cov * dt


def cal_continuation_value_multiasset(
    w1, b1, w2, b2,
    n_hidden_units,
    S_t_prev,        # shape (n_sims, n_assets): raw prices at t_{m-1}
    r,
    sigma_vector,    # shape (n_assets,)
    correlation_mat, # shape (n_assets, n_assets)
    dt,
    normalizer=None
):
    """
    Computes the continuation value:
      Q_{t_{m-1}}(S_t_prev)
      = E[ NN(log(S_{t_m})) | S_{t_{m-1}} ] 
]
    Args:
        w1 : shape (n_assets, n_hidden_units) - first-layer weights
        b1 : shape (n_hidden_units,)         - first-layer biases
        w2 : shape (n_hidden_units,) or (n_hidden_units, 1) - second-layer weights
        b2 : shape (1,) or float             - output bias
        n_hidden_units (int)
        S_t_prev : shape (n_sims, n_assets)
        r (float) : risk-free rate
        sigma_vector : shape (n_assets,)
        correlation_mat : shape (n_assets, n_assets)
        dt (float): t_m - t_{m-1}
        normalizer : If you used normalization, e.g. dividing by S0. 
                     If None, we assume S_t_prev is actual price.
    Returns:
        continuation_val : shape (n_sims,) 
    """
    n_sims, n_assets = S_t_prev.shape
    continuation_val = np.zeros(n_sims)
    cov_mat = build_covariance_matrix_dt(sigma_vector, correlation_mat, dt)

    if len(w2.shape) == 2 and w2.shape[1] == 1:
        w2 = w2.flatten()
    if isinstance(b2, np.ndarray) and b2.shape == (1,):
        b2 = b2[0]

    for i in range(n_sims):
        S_n = S_t_prev[i, :] + 1e-14
        logS_prev = np.log(S_n)
        mu_vec = logS_prev + (r - 0.5*(sigma_vector**2))*dt

        nn_sum = 0.0
        for neuron_index in range(n_hidden_units):
            w1_i = w1.reshape(n_assets, n_hidden_units)[:, neuron_index]
            b1_i = b1[neuron_index]
            meanZ = np.dot(w1_i, mu_vec)
            varZ  = w1_i @ cov_mat @ w1_i
            meanZprime = meanZ + b1_i
            stdZprime  = np.sqrt(varZ) if varZ > 1e-14 else 0.0
            relu_i = conditional_expectation_closed_form(meanZprime, stdZprime)
            nn_sum += w2[neuron_index] * relu_i

        nn_sum += b2
        continuation_val[i] = nn_sum

    return continuation_val

def create_shallow_NN(input_dim, hidden_units):
    """
    Create a shallow neural network with 1 hidden layer

    Args:
        input_dim (int): number of nodes in input layer
        hidden_units (int): number of nodes in hidden layer

    Returns:
        model : Neural network model
    """
    model = Sequential()
    # Use Input layer for specifying input shape
    model.add(Input(shape=(input_dim,)))
    model.add(Dense(hidden_units, activation='relu', kernel_initializer='random_uniform', bias_initializer= 'random_uniform'))
    model.add(Dense(1, activation='linear', kernel_initializer='random_uniform', bias_initializer= 'random_uniform'))
    
    return model


###  Testing continuation value single step

In [ ]:
 
#Generate parameters using the provided functions
 
n_assets = 2
rho = 0.3

# Generate S0, mu, and sigma
S0 = generate_parameters(n_assets, value=100.0, mode='same')
mu = generate_parameters(n_assets, value=0.05,  mode='same')  # Not used as "mu" in your snippet
sigma_gen = generate_parameters(n_assets, value=0.2,  mode='same')  # This yields [0.2, 0.2]

# Build correlation matrix
corr_matrix = build_correlation_matrix(rho, n_assets)

print("S0 =", S0)              
print("mu =", mu)             
print("sigma_gen =", sigma_gen)
print("corr_matrix =\n", corr_matrix)
print("----------------------------------------------------------")

 
# 3) Single-step RLNN test


sigma1, sigma2 = sigma_gen 
d2   = n_assets
r2   = 0.02
rho2 = 0.3
dt2  = 1.0
S_tm1_2 = S0

logS_tm1_2 = np.log(S_tm1_2)
mu2 = np.array([
    logS_tm1_2[0] + (r2 - 0.5*sigma1**2)*dt2,
    logS_tm1_2[1] + (r2 - 0.5*sigma2**2)*dt2
])
Sigma2 = np.array([
    [sigma1**2,          rho2*sigma1*sigma2],
    [rho2*sigma1*sigma2, sigma2**2         ]
]) * dt2

# (B) Monte Carlo
n_sims_mc = 200000
rng = np.random.default_rng(123)
L2 = np.linalg.cholesky(Sigma2)
Z2 = rng.normal(size=(d2, n_sims_mc))
logS_m_samples = (L2 @ Z2).T + mu2

# Single-neuron parameters
w_rlnn = np.array([0.8, 1.2])
b_rlnn = -2.0

Y_sims = logS_m_samples @ w_rlnn + b_rlnn
mc_payoff = np.maximum(Y_sims, 0.0)
mc_value_2 = mc_payoff.mean()


# We'll tile the same S_tm1_2 into an array, so all s0 are same => shape (n_sims_test, d2)
n_sims_test = 5000
S_tm1_array = np.tile(S_tm1_2, (n_sims_test, 1))

w1_matrix = w_rlnn.reshape(d2, 1)  # shape (2,1)
b1_array  = np.array([b_rlnn])     # shape (1,)
w2_array  = np.array([1.0])        # second-layer weight = 1
b2_scalar = 0.0                    # output bias = 0

rlnn_vals = cal_continuation_value_multiasset(
    w1_matrix, b1_array, w2_array, b2_scalar,
    n_hidden_units=1,
    S_t_prev=S_tm1_array,
    r=r2,
    sigma_vector=sigma_gen,
    correlation_mat=corr_matrix, 
    dt=dt2
)

rlnn_val_single = rlnn_vals[0]

print("\n=== Single-Neuron (RLNN) vs. Direct & MC ===")
print(f"Monte Carlo (200k sims) = {mc_value_2:.6f}")
print(f"RLNN-style single neuron= {rlnn_val_single:.6f}")


S0 = [100. 100.]
mu = [0.05 0.05]
sigma_gen = [0.2 0.2]
corr_matrix =
 [[1.  0.3]
 [0.3 1. ]]
----------------------------------------------------------

=== Single-Neuron (RLNN) vs. Direct & MC ===
Monte Carlo (200k sims) = 7.210421
RLNN-style single neuron= 7.210340


# Main RLNN_Algo for Multi-Asset with Log-Price & Closed-Form Continuation


In [9]:
def RLNN_Algo_multiasset(
    n_assets, S0, K, r, q, vol, rho, 
    sample_size, n_steps, no_mon, T, style, 
    no_hidden_units, l_rate,
    build_correlation_matrix,
    generate_parameters,
    simulate_correlated_gbm,
    payoff_fun,
    create_shallow_NN
):
    """
    Multi-Asset RLNN with closed form continuation value
    based on log-prices.

    Args:
        n_assets (int): number of assets
        S0 (float): initial price (if same across assets)
        K (float): strike
        r (float): risk-free rate
        q (float): div yield
        vol (float): base volatility
        rho (float): correlation param
        sample_size (int): number of MC paths
        n_steps (int): total time steps
        no_mon (int): number of monitoring points
        T (float): maturity
        style (str): 'call'/'put'
        no_hidden_units (int): hidden layer size
        l_rate (float): learning rate

        build_correlation_matrix (func): user-provided
        generate_parameters (func): user-provided
        simulate_correlated_gbm (func): user-provided
        payoff_fun (func): payoff for multi-asset
        create_shallow_NN (func): returns a Keras model

    Returns:
        (approx_value_t0, weights_list, errors_list)
    """

    # For storing neural net weights at each step, plus training errors
    weights = []
    errors = []

    # 1) Build correlation & simulate
    corr_matrix = build_correlation_matrix(rho, n_assets)   # shape (n_assets, n_assets)

    S0_vec    = generate_parameters(n_assets, value=S0,  mode='same') 
    sigma_vec = generate_parameters(n_assets, value=vol,   mode='same')

    # shape => (n_sims, n_assets, n_steps+1)
    stock_paths = simulate_correlated_gbm(
        S0_vec, r, q, sigma_vec, corr_matrix, 
        T, n_steps, n_sims=sample_size, random_seed=None
    )

    # 2) Initialize array for option values
    option_price = np.zeros((sample_size, n_steps+1))

    # 3) Compute payoff at maturity
    payoff_T = payoff_fun(stock_paths[:, :, n_steps], K, style)  # shape (n_sims,)
    option_price[:, n_steps] = payoff_T

    # 4) Pre-train a shallow NN at final time
    model = create_shallow_NN(n_assets, no_hidden_units)
    model.compile(loss='mean_squared_error', optimizer=Adam(l_rate))

    X_final = np.log(stock_paths[:, :, n_steps] + 1e-14)  # log-prices
    y_final = payoff_T

    model.fit(
        X_final, y_final,
        epochs=1000,
        batch_size=int(0.1 * sample_size),
        verbose=0
    )

    # Evaluate final-time MSE:
    y_pred_final = model.predict(X_final).flatten()
    final_mse = np.mean((y_pred_final - y_final)**2)
    errors.append(final_mse)

    # Move backwards in time 
    early_stopping = EarlyStopping(
        monitor='val_loss', mode='min',
        patience=20, restore_best_weights=True,
        start_from_epoch=100
    )

    for m in range(n_steps-1, -1, -1):
        # Train 
        X_m = np.log(stock_paths[:, :, m] + 1e-14)   # shape (n_sims, n_assets)
        y_m = option_price[:, m]                    # shape (n_sims,)

        # Split data for validation
        X_train, X_test, y_train, y_test = train_test_split(
            X_m, y_m, test_size=0.2
        )

        model.fit(
            X_train, y_train,
            epochs=1000,
            batch_size=int(0.1 * X_train.shape[0]),
            validation_data=(X_test, y_test),
            callbacks=[early_stopping],
            verbose=0
        )

        # Evaluate MSE for logging
        y_hat_test = model.predict(X_test).flatten()
        test_mse = np.mean((y_hat_test - y_test)**2)
        errors.append(test_mse)

        #  Extract NN weights
        # shape of first layer weights: (n_assets, n_hidden_units)
        w1_ = model.layers[0].get_weights()[0]
        b1_ = model.layers[0].get_weights()[1]
        w2_ = model.layers[1].get_weights()[0]
        b2_ = model.layers[1].get_weights()[1]
        weights.append(model.get_weights())

        # If m>0, compute continuation at time m-1:
        if m > 0:
            S_mminus1 = stock_paths[:, :, m-1]  # shape (n_sims, n_assets)
            fun_h     = payoff_fun(S_mminus1, K, style)   # shape (n_sims,)

            dt = (T / n_steps) 

            # closed-form expectation
            cont_val = cal_continuation_value_multiasset(
                w1_, b1_, w2_, b2_,
                no_hidden_units,
                S_mminus1,    #  price at time (m-1)
                r,           
                sigma_vec,
                corr_matrix,
                dt,
                normalizer=None 
            )

            # final
            option_price[:, m-1] = np.maximum(fun_h, cont_val)

    # Return approximate value at t0, plus weights and errors
    return option_price[:, 0].mean(), weights, errors


In [10]:
n_assets     = 3
S0           = 100.0
K            = 100.0
r            = 0.05
q            = 0.00
vol          = 0.2
rho          = 0.5 
sample_size  = 20000
n_steps      = 30
no_mon       = 4     # passed, but unused in RLNN_Algo_multiasset currently. Left it for when we reimplement Bermudan options
T            = 1.0   # maturity = 1 year
style        = "put"
no_hidden_units = 16
l_rate       = 0.05

# Call the RLNN solver
approx_value_t0, weights_list, errors_list = RLNN_Algo_multiasset(
    n_assets, S0, K, r, q, vol, rho,
    sample_size, n_steps, no_mon, T, style,
    no_hidden_units, l_rate,
    build_correlation_matrix,
    generate_parameters,
    simulate_correlated_gbm,
    payoff_fun,
    create_shallow_NN
)

print(f"Estimated Option Value at t0: {approx_value_t0:.4f}")
print("Final training errors_list:", errors_list)

(20000, 3)
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(20000, 3)
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/